<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Training an Object Detection Model

In this notebook, we give an introduction to training an object detection model using [torchvision](https://pytorch.org/docs/stable/torchvision/index.html). Using a small dataset, we demonstrate how to train and evaluate a FasterRCNN object detection model. We also cover one of the most common ways to store data on a file system for this type of problem.

## Initialization

In [ ]:
# Ensure edits to libraries are loaded and plotting is shown in the notebook.
%reload_ext autoreload
%autoreload 2
%matplotlib inline
# %load_ext blackcellmagic

Import all the functions we need.

In [ ]:
import sys
sys.path.append("../../")

import os
import time
from pathlib import Path
from PIL import Image
from random import randrange
from typing import Tuple, List
from matplotlib import pyplot as plt

import torch
import numpy as np
import cv2
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

from utils_cv.common.data import unzip_url, data_path
from utils_cv.detection.data import Urls
from utils_cv.detection.dataset import DetectionDataset
from utils_cv.detection.plot import display_bounding_boxes
from utils_cv.detection.model import get_bounding_boxes, get_transform, get_pretrained_fasterrcnn
from utils_cv.detection.references.engine import train_one_epoch, evaluate
from utils_cv.detection.references.utils import collate_fn
from utils_cv.common.gpu import which_processor

which_processor()

This shows your machine's GPUs (if has any) and the computing device `torch/torchvision` is using. We suggest using an  [Azure DSVM](https://azure.microsoft.com/en-us/services/virtual-machines/data-science-virtual-machines/) Standard NC6 as a GPU compute resource.

Next, set some model runtime parameters. We use the `unzip_url` helper function to download and unzip the data used in this example notebook.

In [ ]:
DATA_PATH     = unzip_url(Urls.fridge_objects_path, exist_ok=True)
EPOCHS        = 3
LEARNING_RATE = 0.005
# IM_SIZE       = 300
# BATCH_SIZE    = 16
# ARCHITECTURE  = models.resnet18

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

---

# Prepare Object Detection Dataset

In this notebook, we use a toy dataset called *Fridge Objects*, which consists of 134 images of 4 classes of beverage container `{can, carton, milk bottle, water bottle}` photos taken on different backgrounds. The helper function downloads and unzips data set to the `ComputerVision/data` directory.

Set that directory in the `path` variable for ease of use throughout the notebook.

In [ ]:
path = Path(DATA_PATH)
os.listdir(path)

You'll notice that we have two different folders inside:
- `/images`
- `/annotations`

This format for object detection is fairly common.

```
/data
+-- images
|   +-- image1.jpg
|   +-- image2.jpg
|   +-- ...
+-- annotations
|   +-- image1.xml
|   +-- image2.xml
|   +-- ...
+-- ...
```

The xml files inside the annotation folder contain information on where the objects are the xml's corresponding image file. In this example, our fridge object dataset is annotated in Pascal VOC format. This is one of the most common formats for labelling object detection datasets.

```xml
<!-- Example Pascal VOC annotation -->
<annotation>
    <folder>images</folder>
    <filename>1.jpg</filename>
    <path>../images/1.jpg</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>499</width>
        <height>666</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>

    <object>
        <name>carton</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>100</xmin>
            <ymin>173</ymin>
            <xmax>233</xmax>
            <ymax>521</ymax>
        </bndbox>
    </object>
</annotation>
```

You'll notice that inside the annotation xml file, we can see which image the file references `<path>`, the number of `<objects>` in the image, that the image is of (`<name>`) and the bounding box of that object (`<bndbox>`).

# Load Images

To load the data, we need to create a Dataset object class that Torchvision knows how to use. In short, we'll need to create a class and implement the `__getitem__` method. More information here: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#defining-the-dataset

To make it more convinient, we've created a `DetectionDataset` class that knows how to extract annotation information from the Pascal VOC format and meet the requirements of the Torchvision dataset object class. 

In [ ]:
fridge_objects_dataset = DetectionDataset(DATA_PATH)

Lets visualize some of the images to make sure the annotation looks correct.

In [ ]:
fridge_objects_dataset.show_batch(rows=2)

Now that we have our dataset setup, we want to split it into train/test sets, and then use them to create our data loaders which will be used when fine-tuning the model.

In [ ]:
# split the dataset in train and test set
train_ds, test_ds = fridge_objects_dataset.split_train_test()

# define training and validation data loaders
train_dl = DataLoader(
    train_ds, batch_size=2, shuffle=True, num_workers=4, collate_fn=collate_fn
)

test_dl = DataLoader(
    test_ds, batch_size=1, shuffle=False, num_workers=4, collate_fn=collate_fn
)

Lets make sure that the number of images in the training and testing set is what is expected.

In [ ]:
print(f"""\
Training images: {len(train_ds)}
Testing images: {len(test_ds)}\
""")

# Fine Tune a Pretrained Model

For this object detector, we use FasterRCNN, and Stochastic Gradient Descent as our optimizer. Our FasterRCNN model is pretrained on COCO, a large-scale object detection, segmentation, and captioning dataset that contains over 200K labeled images with over 80 label cateogories.

In [ ]:
# get the model using our helper function
model = get_pretrained_fasterrcnn(len(fridge_objects_dataset.categories)).to(device)

# construct our optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=LEARNING_RATE, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

We can now fine-tune the model using our training data loader (`train_dl`) and evaluate our results using our testing data loader (`test_dl`).

In [ ]:
import math

def warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor):
    def f(x):
        if x >= warmup_iters:
            return 1
        alpha = float(x) / warmup_iters
        return warmup_factor * (1 - alpha) + alpha

    return torch.optim.lr_scheduler.LambdaLR(optimizer, f)


def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = 1 #get_world_size()
    if world_size < 2:
        return input_dict
    with torch.no_grad():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict

def train_one_epoch2(model, optimizer, lr_scheduler, data_loader, device, epoch, print_freq):
    model.train()
    header = "Epoch: [{}]".format(epoch)
    all_losses = []

    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(
            optimizer, warmup_iters, warmup_factor
        )

    for idx, (images, targets) in enumerate(data_loader):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()
            
        all_losses.append(losses_reduced.item())

    return all_losses

In [ ]:
for epoch in range(EPOCHS):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, lr_scheduler, train_dl, device, epoch, print_freq=10)
    
    # update the learning rate
    lr_scheduler.step()

In [ ]:
e = evaluate(model, test_dl, device=device)

In [ ]:
e.summarize()

### Inference

In [ ]:
# select image
im_path = test_ds.dataset.root/'images'/test_ds.dataset.ims[randrange(len(test_ds))]
im = Image.open(im_path)

# Define PyTorch Transform
transform = transforms.Compose([transforms.ToTensor()])

# Apply the transform to the image
im = transform(im).cuda()

In [ ]:
# put the model in evaluation mode
model.eval()

with torch.no_grad():
    start = time.time()
    pred = model([im])
    print(f"Time spend: {time.time() - start}")

In [ ]:
pred_labels, pred_boxes = get_bounding_boxes(pred)
display_bounding_boxes(pred_boxes, [fridge_objects_dataset.categories[l] for l in pred_labels], im_path)

In [ ]:
fridge_objects_dataset.categories